# Complex Analysis Foundations

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** Days 113-140
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
from matplotlib.patches import Circle
from numpy.linalg import eig, norm, det, inv
import cmath
%matplotlib inline

# Publication-quality plot defaults
plt.rcParams.update({
    'figure.figsize': (8, 6),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'lines.linewidth': 2,
    'figure.dpi': 100,
})
print("Imports loaded. Ready for complex analysis explorations.")

## Why Complex Numbers are Essential for Quantum Mechanics

Quantum mechanics is **inherently complex-valued**. The Schrodinger equation

$$i\hbar \frac{\partial}{\partial t}|\psi\rangle = \hat{H}|\psi\rangle$$

contains $i = \sqrt{-1}$ explicitly. This is not a mathematical convenience ---
recent experiments (2021-2022) have confirmed that **real-valued quantum theory
cannot reproduce all quantum predictions**.

**Complex numbers provide:**
- **Phase information** --- interference depends on relative phases
- **Unitary evolution** --- $U = e^{-iHt/\hbar}$ requires complex exponentials
- **Probability amplitudes** --- $\psi(x) \in \mathbb{C}$, with $|\psi(x)|^2$ giving probability

This notebook builds fluency with complex arithmetic, Euler's formula,
complex function mappings, and the linear algebra structures (Hermitian, unitary
matrices) that form the mathematical backbone of quantum theory.

## 1. Complex Numbers on the Argand Plane

A complex number $z = a + bi$ can be represented as a point $(a, b)$ in the
**Argand plane** (complex plane). Equivalently, in polar form:

$$z = r e^{i\theta} = r(\cos\theta + i\sin\theta)$$

where $r = |z| = \sqrt{a^2 + b^2}$ is the **modulus** and
$\theta = \arg(z) = \text{atan2}(b, a)$ is the **argument**.

In [ ]:
# --- Complex number visualization on the Argand plane ---

# Define several complex numbers
z_list = [
    (2 + 3j,   'z_1 = 2+3i'),
    (-1 + 2j,  'z_2 = -1+2i'),
    (3 - 1j,   'z_3 = 3-i'),
    (-2 - 2j,  'z_4 = -2-2i'),
]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: individual complex numbers
ax = axes[0]
for z, label in z_list:
    ax.annotate('', xy=(z.real, z.imag), xytext=(0, 0),
                arrowprops=dict(arrowstyle='->', lw=2, color='steelblue'))
    ax.plot(z.real, z.imag, 'o', markersize=8, color='darkred')
    ax.annotate(label, xy=(z.real, z.imag), xytext=(5, 5),
                textcoords='offset points', fontsize=10)

ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linewidth=0.5)
ax.axvline(0, color='k', linewidth=0.5)
ax.set_xlabel('Re(z)')
ax.set_ylabel('Im(z)')
ax.set_title('Complex Numbers on the Argand Plane')

# Right: polar form visualization
ax = axes[1]
z = 3 + 2j
r = abs(z)
theta = cmath.phase(z)

# Draw the vector
ax.annotate('', xy=(z.real, z.imag), xytext=(0, 0),
            arrowprops=dict(arrowstyle='->', lw=2.5, color='darkblue'))
ax.plot(z.real, z.imag, 'o', markersize=10, color='red')

# Draw the angle arc
arc_theta = np.linspace(0, theta, 50)
arc_r = 0.8
ax.plot(arc_r * np.cos(arc_theta), arc_r * np.sin(arc_theta), 'g-', linewidth=2)
ax.annotate(f'$\\theta = {np.degrees(theta):.1f}°$',
            xy=(0.9, 0.3), fontsize=12, color='green')

# Draw projections
ax.plot([z.real, z.real], [0, z.imag], 'k--', alpha=0.4)
ax.plot([0, z.real], [z.imag, z.imag], 'k--', alpha=0.4)

ax.annotate(f'$z = {z.real:.0f}+{z.imag:.0f}i$\n$r = |z| = {r:.3f}$',
            xy=(z.real + 0.1, z.imag + 0.2), fontsize=11)

ax.set_xlim(-1, 5)
ax.set_ylim(-1, 4)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linewidth=0.5)
ax.axvline(0, color='k', linewidth=0.5)
ax.set_xlabel('Re(z)')
ax.set_ylabel('Im(z)')
ax.set_title('Polar Form: $z = re^{i\\theta}$')

plt.tight_layout()
plt.show()

# Print properties
print("Complex number properties:")
for z, label in z_list:
    print(f"  {label}: |z| = {abs(z):.3f}, arg(z) = {np.degrees(cmath.phase(z)):.1f} deg, "
          f"conjugate = {z.conjugate()}")

## 2. Euler's Formula: $e^{i\theta} = \cos\theta + i\sin\theta$

This is arguably the most important formula connecting algebra, geometry,
and analysis. It shows that:

- **Complex exponentials trace the unit circle** as $\theta$ varies
- **Multiplication by $e^{i\theta}$** is a rotation by angle $\theta$
- Setting $\theta = \pi$: $e^{i\pi} + 1 = 0$ (Euler's identity)

**QM Connection:** Time evolution of an energy eigenstate:
$$|\psi(t)\rangle = e^{-iEt/\hbar}|E\rangle$$
The state rotates in the complex plane with angular frequency $\omega = E/\hbar$.

In [ ]:
# --- Euler's formula visualization: e^{i theta} on the unit circle ---

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: unit circle traced by e^{i*theta}
ax = axes[0]
theta = np.linspace(0, 2 * np.pi, 500)
z_circle = np.exp(1j * theta)
ax.plot(z_circle.real, z_circle.imag, 'b-', linewidth=2, label='$e^{i\\theta}$')

# Mark special angles
special = {
    0: '$1$', np.pi/6: '$e^{i\\pi/6}$', np.pi/4: '$e^{i\\pi/4}$',
    np.pi/3: '$e^{i\\pi/3}$', np.pi/2: '$i$',
    np.pi: '$-1$', 3*np.pi/2: '$-i$'
}
for ang, label in special.items():
    z = np.exp(1j * ang)
    ax.plot(z.real, z.imag, 'ro', markersize=8)
    offset = (10, 10) if ang < np.pi else (-30, -15)
    ax.annotate(label, xy=(z.real, z.imag), xytext=offset,
                textcoords='offset points', fontsize=10)

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linewidth=0.5)
ax.axvline(0, color='k', linewidth=0.5)
ax.set_xlabel('Re')
ax.set_ylabel('Im')
ax.set_title("Euler's Formula: $e^{i\\theta}$ traces the unit circle")

# Right: cos and sin as real/imaginary parts
ax = axes[1]
theta_range = np.linspace(0, 4 * np.pi, 500)
z_evolving = np.exp(1j * theta_range)

ax.plot(theta_range, z_evolving.real, 'b-', linewidth=2, label='$\cos(\\theta) = \mathrm{Re}(e^{i\\theta})$')
ax.plot(theta_range, z_evolving.imag, 'r-', linewidth=2, label='$\sin(\\theta) = \mathrm{Im}(e^{i\\theta})$')
ax.set_xlabel('$\\theta$ (radians)')
ax.set_ylabel('Value')
ax.set_title('Real and Imaginary parts of $e^{i\\theta}$')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 4 * np.pi)

plt.tight_layout()
plt.show()

# Verify Euler's identity
euler_identity = np.exp(1j * np.pi) + 1
print(f"Euler's identity: e^(i*pi) + 1 = {euler_identity:.2e}  (should be ~0)")
print(f"Verification: e^(i*pi/4) = {np.exp(1j*np.pi/4):.6f}")
print(f"Expected:     cos(pi/4) + i*sin(pi/4) = {np.cos(np.pi/4):.6f} + {np.sin(np.pi/4):.6f}i")

## 3. Complex Function Mappings with Domain Coloring

A function $f: \mathbb{C} \to \mathbb{C}$ maps every point in one complex plane
to another. **Domain coloring** visualizes this by coloring the input plane:
- **Hue** encodes the argument (phase) of $f(z)$
- **Brightness** encodes the modulus $|f(z)|$

We explore three fundamental mappings:
- $f(z) = z^2$ --- doubles angles, squares distances
- $f(z) = 1/z$ --- inversion (maps inside/outside the unit circle)
- $f(z) = e^z$ --- maps vertical lines to circles, horizontal lines to rays

In [ ]:
# --- Domain coloring for complex functions ---

def domain_coloring(f, extent=(-2, 2, -2, 2), N=500, title="f(z)"):
    """
    Domain coloring visualization of a complex function f.
    Hue = arg(f(z)), Brightness = |f(z)| (compressed with log).
    """
    x = np.linspace(extent[0], extent[1], N)
    y = np.linspace(extent[2], extent[3], N)
    X, Y = np.meshgrid(x, y)
    Z = X + 1j * Y

    # Evaluate function (handle division by zero gracefully)
    with np.errstate(divide='ignore', invalid='ignore'):
        W = f(Z)

    # Hue from argument (0 to 1)
    H = (np.angle(W) + np.pi) / (2 * np.pi)

    # Saturation: constant
    S = np.ones_like(H) * 0.9

    # Value (brightness) from modulus, compressed with log
    modulus = np.abs(W)
    V = 1 - 1 / (1 + modulus**0.3)  # Smooth compression

    # Handle NaN/inf
    H = np.nan_to_num(H, nan=0.0)
    V = np.nan_to_num(V, nan=0.5)

    # Convert HSV to RGB
    HSV = np.stack([H, S, V], axis=-1)
    RGB = hsv_to_rgb(HSV)

    return RGB, extent


# Create domain coloring for three functions
functions = [
    (lambda z: z**2,     "$f(z) = z^2$",     (-2, 2, -2, 2)),
    (lambda z: 1/z,      "$f(z) = 1/z$",     (-2, 2, -2, 2)),
    (lambda z: np.exp(z), "$f(z) = e^z$",     (-3, 3, -3, 3)),
]

fig, axes = plt.subplots(1, 3, figsize=(18, 5.5))

for ax, (f, title, extent) in zip(axes, functions):
    RGB, ext = domain_coloring(f, extent=extent, title=title)
    ax.imshow(RGB, extent=ext, origin='lower', aspect='equal')
    ax.set_xlabel('Re(z)')
    ax.set_ylabel('Im(z)')
    ax.set_title(title, fontsize=14)

    # Add unit circle for reference
    theta = np.linspace(0, 2 * np.pi, 100)
    ax.plot(np.cos(theta), np.sin(theta), 'w--', alpha=0.5, linewidth=1)

plt.suptitle('Domain Coloring of Complex Functions', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()
print("Hue = phase of f(z), Brightness = |f(z)| (compressed)")
print("White dashed circle = unit circle |z| = 1")

## 4. Inner Products in Complex Vector Spaces

In a complex vector space, the inner product is **sesquilinear**:

$$\langle u, v \rangle = \sum_i \overline{u_i} \, v_i = u^\dagger v$$

Note the **conjugation** on the first argument (physics convention). Key properties:
- $\langle u, v \rangle = \overline{\langle v, u \rangle}$ (conjugate symmetry)
- $\langle u, u \rangle \geq 0$ with equality iff $u = 0$ (positive definiteness)
- $\langle u, \alpha v + \beta w \rangle = \alpha\langle u, v\rangle + \beta\langle u, w\rangle$ (linearity in second argument)

**QM Connection:** The probability amplitude for transitioning from state $|\psi\rangle$
to $|\phi\rangle$ is $\langle\phi|\psi\rangle$, and the probability is $|\langle\phi|\psi\rangle|^2$.

In [ ]:
# --- Complex inner products and orthogonality ---

# Define complex vectors (qubit states)
# |+> and |-> states (Hadamard basis)
ket_plus  = np.array([1, 1], dtype=complex) / np.sqrt(2)
ket_minus = np.array([1, -1], dtype=complex) / np.sqrt(2)
# |+i> and |-i> states (Y basis)
ket_plus_i  = np.array([1, 1j], dtype=complex) / np.sqrt(2)
ket_minus_i = np.array([1, -1j], dtype=complex) / np.sqrt(2)

states = {
    '|+>': ket_plus,
    '|->': ket_minus,
    '|+i>': ket_plus_i,
    '|-i>': ket_minus_i,
}

print("=== Complex Inner Products <phi|psi> ===\n")
print(f"{'':8s}", end='')
for name in states:
    print(f"{name:>12s}", end='')
print()

for name_i, psi in states.items():
    print(f"{name_i:8s}", end='')
    for name_j, phi in states.items():
        # Inner product: <phi|psi> = phi^dagger @ psi
        ip = np.vdot(phi, psi)  # np.vdot conjugates the first argument
        print(f"{ip.real:+.3f}{ip.imag:+.3f}i", end='  ')
    print()

print("\n=== Orthogonality checks ===")
print(f"<+|-> = {np.vdot(ket_plus, ket_minus):.6f}  (orthogonal: should be 0)")
print(f"<+i|-i> = {np.vdot(ket_plus_i, ket_minus_i):.6f}  (orthogonal: should be 0)")
print(f"<+|+i> = {np.vdot(ket_plus, ket_plus_i):.6f}  (NOT orthogonal)")

print("\n=== Norm (should all be 1 for normalized states) ===")
for name, v in states.items():
    print(f"  ||{name}|| = {norm(v):.6f}")

## 5. Hermitian and Unitary Matrices

Two classes of matrices are central to quantum mechanics:

**Hermitian matrices** ($A = A^\dagger$):
- Eigenvalues are **real** (measurement outcomes)
- Eigenvectors for distinct eigenvalues are **orthogonal**
- Represent **observables** in QM

**Unitary matrices** ($UU^\dagger = U^\dagger U = I$):
- Eigenvalues have **unit modulus** ($|\lambda| = 1$)
- Preserve inner products: $\langle U\psi | U\phi \rangle = \langle \psi | \phi \rangle$
- Represent **time evolution** and **quantum gates**

**Spectral Theorem:** Every Hermitian matrix can be diagonalized by a unitary:
$$A = U \Lambda U^\dagger, \qquad \Lambda = \text{diag}(\lambda_1, \dots, \lambda_n)$$

In [ ]:
# --- Hermitian and Unitary matrix properties ---

# Construct a Hermitian matrix
H = np.array([
    [2,    1+1j, 0   ],
    [1-1j, 3,    2-1j],
    [0,    2+1j, 1   ]
], dtype=complex)

print("=== Hermitian Matrix H ===")
print(H)
print(f"\nIs Hermitian (H = H^dag)? {np.allclose(H, H.conj().T)}")

# Eigendecomposition
eigenvalues, eigenvectors = eig(H)
print(f"\nEigenvalues: {np.round(eigenvalues.real, 6)}")
print(f"Are eigenvalues real? Max imaginary part: {np.max(np.abs(eigenvalues.imag)):.2e}")

# Check orthogonality of eigenvectors
print("\nEigenvector orthogonality (should be ~identity):")
overlap = eigenvectors.conj().T @ eigenvectors
print(np.round(overlap, 6))

# Construct a Unitary matrix from a Hermitian (U = e^{iH})
# For small matrices, use eigendecomposition
U = eigenvectors @ np.diag(np.exp(1j * eigenvalues.real)) @ eigenvectors.conj().T
print("\n=== Unitary Matrix U = e^{iH} ===")
print(np.round(U, 4))
print(f"\nIs Unitary (UU^dag = I)? {np.allclose(U @ U.conj().T, np.eye(3))}")

# Eigenvalues of unitary should have |lambda| = 1
u_vals = eig(U)[0]
print(f"\nEigenvalues of U: {np.round(u_vals, 4)}")
print(f"|lambda| values:  {np.round(np.abs(u_vals), 6)}")
print(f"All unit modulus?  {np.allclose(np.abs(u_vals), 1.0)}")

In [ ]:
# --- Spectral Theorem Demonstration ---
# For Hermitian A: A = sum_i lambda_i |v_i><v_i|

A = np.array([
    [5, 2-1j],
    [2+1j, 3]
], dtype=complex)

print("Hermitian matrix A:")
print(A)
print(f"Is Hermitian? {np.allclose(A, A.conj().T)}")

# Eigendecomposition
eigenvalues, eigenvectors = eig(A)
print(f"\nEigenvalues: {np.round(eigenvalues.real, 6)}")

# Spectral decomposition: A = sum lambda_i * |v_i><v_i|
A_reconstructed = np.zeros_like(A)
print("\n--- Spectral Decomposition ---")
for i in range(len(eigenvalues)):
    lam = eigenvalues[i].real
    v = eigenvectors[:, i:i+1]  # Column vector
    projector = v @ v.conj().T  # |v><v| (outer product)
    A_reconstructed += lam * projector
    print(f"\n  lambda_{i} = {lam:.4f}")
    print(f"  |v_{i}> = {v.flatten().round(4)}")
    print(f"  |v_{i}><v_{i}| =")
    print(f"  {np.round(projector, 4)}")

print(f"\nReconstruction: A = sum lambda_i |v_i><v_i|")
print(f"Reconstruction error: {norm(A - A_reconstructed):.2e}")

# Projectors are idempotent and complete
P0 = eigenvectors[:, 0:1] @ eigenvectors[:, 0:1].conj().T
P1 = eigenvectors[:, 1:2] @ eigenvectors[:, 1:2].conj().T
print(f"\nP0^2 = P0? {np.allclose(P0 @ P0, P0)}")
print(f"P1^2 = P1? {np.allclose(P1 @ P1, P1)}")
print(f"P0 + P1 = I? {np.allclose(P0 + P1, np.eye(2))}")
print(f"P0 P1 = 0? {np.allclose(P0 @ P1, np.zeros((2,2)))}")

## Summary

| Concept | Key Property | QM Significance |
|---------|-------------|-----------------|
| Complex numbers | $z = re^{i\theta}$ | Probability amplitudes |
| Euler's formula | $e^{i\theta} = \cos\theta + i\sin\theta$ | Time evolution phases |
| Inner product | $\langle u,v\rangle = u^\dagger v$ | Transition amplitudes |
| Hermitian matrix | $A = A^\dagger$, real eigenvalues | Observables |
| Unitary matrix | $UU^\dagger = I$, $|\lambda|=1$ | Quantum gates, evolution |
| Spectral theorem | $A = \sum \lambda_i |v_i\rangle\langle v_i|$ | Measurement theory |

**Key Takeaways:**
1. Complex numbers are not optional in QM --- they are structurally necessary
2. Euler's formula unifies exponentials, trigonometry, and rotation
3. Domain coloring reveals the rich geometry of complex functions
4. Hermitian matrices guarantee real measurement outcomes
5. Unitary matrices preserve probability (norm) during evolution
6. The spectral theorem is the mathematical core of quantum measurement

---
*Next: Month 06 bridges classical and quantum mechanics via Lagrangian and Hamiltonian formulations.*